In [ ]:
from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images_from_PIL
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

import PIL.Image 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

import json

Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


/home/soliverosb/3dStuff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/soliverosb/dust3r/dust3r/cloud_opt/base_opt.py:275: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


In [7]:
import json

with open('/home/soliverosb/OWL/src/external_repositories/data/apple/frame_annotations.js', 'r') as file:
    annots = json.load(file)

In [13]:
annots.sort(key=lambda x: x['frame_number'])

In [14]:
annots[0]

{'sequence_name': '540_79043_153212',
 'frame_number': 1,
 'frame_timestamp': 0.0,
 'image': {'path': 'apple/540_79043_153212/images/frame000001.jpg',
  'size': [900, 2000]},
 'depth': {'path': 'apple/540_79043_153212/depths/frame000001.jpg.geometric.png',
  'scale_adjustment': 1.0,
  'mask_path': 'apple/540_79043_153212/depth_masks/frame000001.png'},
 'mask': {'path': 'apple/540_79043_153212/masks/frame000001.png',
  'mass': 222916.0},
 'viewpoint': {'R': [[-0.9970602989196777,
    -0.07412637025117874,
    0.019390851259231567],
   [0.07354791462421417, -0.9968695640563965, -0.02901434898376465],
   [0.021480876952409744, -0.02750289812684059, 0.9993909001350403]],
  'T': [-0.6326847672462463, 1.212049126625061, 14.048179626464844],
  'focal_length': [3.922766923904419, 3.923976182937622],
  'principal_point': [-0.0, -0.0011111111380159855],
  'intrinsics_format': 'ndc_isotropic'},
 'meta': {'frame_type': 'test_known',
  'frame_splits': ['singlesequence_apple_test_0_known'],
  'eval_

In [2]:
!pwd

/home/soliverosb/dust3r


In [ ]:
device = 'cuda:0'
schedule = 'cosine'
lr = 0.01
niter = 300

model_name = "naver/DUSt3R_ViTLarge_BaseDecoder_512_dpt"

model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

imageDf = pd.read_csv("/home/soliverosb/dataFor3D/carScene.csv")
n = 30
PILImages = [ PIL.Image.open(imageDf['paths'].iloc[i]) for i in range(n) ]
transformationMatrices = torch.tensor([ast.literal_eval(imageDf['rotation_matrices'].iloc[i]) for i in range(n)])

images = load_images_from_PIL(PILImages, size=512)

 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resolution 800x800 --> 512x384
 - changing image with resol

In [66]:
pairs = make_pairs(images, scene_graph='complete', prefilter='cyc20', symmetrize=True)
output = inference(pairs, model, device, batch_size=16)


scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer)
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

# retrieve useful values from scene:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses().cpu().detach()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()

>> Inference with model on 870 image pairs


  0%|          | 0/55 [00:00<?, ?it/s]/home/soliverosb/dust3r/dust3r/inference.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=bool(use_amp)):
/home/soliverosb/dust3r/dust3r/model.py:205: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/soliverosb/dust3r/dust3r/inference.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
100%|██████████| 55/55 [01:30<00:00,  1.65s/it]


 init edge (12*,23*) score=np.float64(16.18464469909668)
 init edge (23,24*) score=np.float64(14.476905822753906)
 init edge (23,10*) score=np.float64(13.846390724182129)
 init edge (14*,24) score=np.float64(13.77676773071289)
 init edge (23,25*) score=np.float64(13.392385482788086)
 init edge (21*,24) score=np.float64(13.24314022064209)
 init edge (13*,24) score=np.float64(13.002606391906738)
 init edge (28*,25) score=np.float64(12.237759590148926)
 init edge (22*,12) score=np.float64(11.685663223266602)
 init edge (22,8*) score=np.float64(10.564742088317871)
 init edge (28,26*) score=np.float64(10.49522876739502)
 init edge (7*,24) score=np.float64(9.887109756469727)
 init edge (20*,23) score=np.float64(9.643139839172363)
 init edge (13,11*) score=np.float64(9.519482612609863)
 init edge (1*,12) score=np.float64(8.512975692749023)
 init edge (2*,23) score=np.float64(8.185733795166016)
 init edge (12,16*) score=np.float64(6.656954288482666)
 init edge (13,9*) score=np.float64(6.651501

100%|██████████| 300/300 [02:26<00:00,  2.05it/s, lr=1.27413e-06 loss=0.00431767]


In [50]:
def getReferenceFrameTransform(X, Y):
    mu_x = torch.mean(X, dim=0)
    mu_y = torch.mean(Y, dim=0)

    var_x = torch.square(X - mu_x).sum(dim=1).mean()
    var_y = torch.square(Y - mu_y).sum(dim=1).mean()
    
    cov_XY = torch.mm((Y - mu_y).T,(X - mu_x))/len(X)

    U, D, Vh = torch.linalg.svd(cov_XY)

    S = torch.eye(X.shape[1])

    if X.shape[1]-1 < torch.linalg.matrix_rank(cov_XY):
        if torch.linalg.det(cov_XY) < 0:
            S[-1, -1] = -1
    else: 
        if torch.linalg.det(U) * torch.linalg.det(Vh) < 0:
            S[-1, -1] = -1
    
    c = 1/var_x*torch.trace(torch.diag(D)@S)
    R = U @ S @ Vh
    t = mu_y - c*R@mu_x

    return R, t, c

In [51]:
def getAllCombinations(batch):
    indices_i = []
    indices_j = []
    batch_size = len(batch)
    for i in range(batch_size):
        for j in range(i + 1, batch_size):
            indices_i.append(i)
            indices_j.append(j)

    indices_i = torch.tensor(indices_i)
    indices_j = torch.tensor(indices_j)

    tensors_i = batch[indices_i]
    tensors_j = batch[indices_j]

    return tensors_i, tensors_j

In [52]:
def calcRTA(positionPreds, positionGt, tau=15):

    positionsPreds_i, positionsPreds_j = getAllCombinations(positionPreds)
    positionsGt_i, positionsGt_j = getAllCombinations(positionGt)

    positionDifferencesPreds = positionsPreds_j - positionsPreds_i
    positionDifferencesGt = positionsGt_j - positionsGt_i

    positionDifferencesPreds = torch.nn.functional.normalize(positionDifferencesPreds)
    positionDifferencesGt = torch.nn.functional.normalize(positionDifferencesGt)

    positionDotDifferences = torch.sum(positionDifferencesPreds*positionDifferencesGt, dim=1)
    positionDotAngles = torch.rad2deg(torch.acos(positionDotDifferences))

    RTA = torch.where(positionDotAngles < tau, 1.0, 0.0).mean()

    return RTA

In [53]:
def calcRRA(rotationPreds, rotationGt, tau=15):
    rotationPreds_i, rotationPreds_j = getAllCombinations(rotationPreds)
    rotationGt_i, rotationGt_j = getAllCombinations(rotationGt)

    rotationPreds_ij = torch.matmul(rotationPreds_i, rotationPreds_j.transpose(1,2))
    rotationGt_ij = torch.matmul(rotationGt_i, rotationGt_j.transpose(1,2))
    
    rotationGtPred_ij = torch.matmul(rotationGt_ij.transpose(1,2), rotationPreds_ij)

    traces = torch.einsum('bii->b', rotationGtPred_ij)

    rotationAngles_ij = torch.rad2deg(torch.acos((traces -1)/2))

    RRA = torch.where(rotationAngles_ij < tau, 1.0, 0.0).mean()

    return RRA


In [56]:
def calcError(HPreds, HGt):
    positionPreds = HPreds[:, :3, 3]
    positionGt = HGt[:, :3, 3]

    rotationPreds = HPreds[:, :3, :3]
    rotationGt = HGt[:, :3, :3]

    
    R, t, c = getReferenceFrameTransform(positionPreds, positionGt)

    positionPredsAligned = c*torch.matmul(positionPreds, R.T) + t
    rotationPredsAligned = torch.matmul(R, rotationPreds)

    RTA = calcRTA(positionPredsAligned, positionGt)

    RRA = calcRRA(rotationPredsAligned, rotationGt)
    
    return RTA, RRA
    

calcError(poses, transformationMatrices)

(tensor(0.9862), tensor(0.9977))

In [67]:
calcError(poses, transformationMatrices)

(tensor(0.9862), tensor(0.9931))